In [1]:
from sklearn import datasets
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Layer, InputSpec, ReLU, LeakyReLU
from tensorflow.keras import initializers, regularizers
from tensorflow.keras.callbacks import Callback
import csv
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
class RadialSoftmax(Layer):

    def __init__(self, c_initializer=initializers.Zeros(), b_initializer=initializers.Zeros(),
                 c_trainable=True, b_trainable=True, **kwargs):
        super(RadialSoftmax, self).__init__(**kwargs)
        self.c_initializer = c_initializer
        self.b_initializer = b_initializer
        self.supports_masking = True
        self.c_trainable = c_trainable
        self.b_trainable = b_trainable

    def build(self, input_shape):
        self.c = self.add_weight(name="c",
                                 shape=(input_shape[1],),
                                 initializer=self.c_initializer,
                                 trainable=self.c_trainable)  # Initialiseerida c ühtedeks / nullideks
        self.b = self.add_weight(name="b",
                                 shape=(input_shape[1],),
                                 initializer=self.b_initializer,
                                 trainable=self.b_trainable)  # Initialiseerida b nullideks
        super(RadialSoftmax, self).build(input_shape)

    def call(self, inputs):
        first_exp = tf.exp(self.c - tf.abs(inputs))

        p = (first_exp + tf.exp(self.b)) / tf.reduce_sum(first_exp + tf.exp(self.b), axis=1, keepdims=True)

        return p

    def compute_output_shape(self, input_shape):
        return input_shape

    def get_config(self):
        return {'c_initializer': self.c_initializer, 'b_initializer': self.b_initializer,
                'c_trainable': self.c_trainable, 'b_trainable': self.b_trainable}

In [3]:
class CustomHistory(Callback):
    def on_train_begin(self, logs=None):
        self.epoch = []
        self.history = {}

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        #self.epoch.append(epoch)
        weights = self.model.layers[-1].get_weights()
        #for k, v in logs.items():
        #    self.history.setdefault(k, []).append(v)
        #self.history.setdefault('c', []).append(weights[0].tolist())
        #self.history.setdefault('b', []).append(weights[1].tolist())
        logs.setdefault('c', []).extend(weights[0].tolist())
        logs.setdefault('b', []).extend(weights[1].tolist())

In [31]:
model = Sequential([
    Dense(100,input_shape=(2,)),
    Activation('relu'),
    #LeakyRelU(),
    Dense(100),
    Activation('relu'),
    #LeakyRelU(),
    Dense(2),
    RadialSoftmax(),
    #Activation('softmax')
])

optimizer = Adam(learning_rate=0.01)

In [ ]:
radial = True # Initialise at False if using regular softmax

In [44]:
def plot(model, plot_min, plot_max, max_prob, name, n_iters, MAct, classes=2):
    """
    Based on Hein, M. et al. :https://github.com/max-andr/relu_networks_overconfident/blob/master/analysis.py
    """
    n_grid = 200
    x_plot = np.linspace(plot_min, plot_max, n_grid)
    y_plot = np.linspace(plot_min, plot_max, n_grid)
    
    points = []
    for xx in x_plot:
        for yy in y_plot:
            points.append((yy, xx))
    points = np.array(points)
    
    if MAct:
        probs = model(points).numpy()
    else:
        logits = model(points) # For not MAct
        probs = tf.nn.softmax(logits).numpy()
    if max_prob:
        z_plot = probs.max(1)
    else:
        z_plot = probs[:, 0]
    z_plot = z_plot.reshape(len(x_plot), len(y_plot)) * 100
    
    ax = plt.gca()
    
    vmax = 100
    vmin = 50 if max_prob else 0
    if classes == 8:
        vmin = 10
        plt.contourf(x_plot, y_plot, z_plot, levels=np.linspace(10, 100, 90))
        cbar = plt.colorbar(ticks=np.linspace(vmin, vmax, 10))

        cbar.ax.set_title('confidence', fontsize=12, pad=12)
        cbar.set_ticklabels(['10%', '20%', '30%', '40%', '50%', '60%', '70%', '80%', '90%', '100%'])
    else:
        plt.contourf(x_plot, y_plot, z_plot, levels=np.linspace(50, 100, 50))
        cbar = plt.colorbar(ticks=np.linspace(vmin, vmax, 6))

        cbar.ax.set_title('confidence', fontsize=12, pad=12)
        cbar.set_ticklabels(['50%', '60%', '70%', '80%', '90%', '100%'])
    
    y_np = np.array(y)
    X0 = X[y_np.argmax(1)==0]
    X1 = X[y_np.argmax(1)==1]
    plt.scatter(X0[:, 0], X0[:, 1], s=20, edgecolors='red', facecolor='None',
                    marker='o', linewidths=0.2)
    plt.scatter(X1[:, 0], X1[:, 1], s=20, edgecolors='green', facecolor='None',
                    marker='s', linewidths=0.2)
    if classes == 8:
        X2 = X[y_np.argmax(1)==2]
        X3 = X[y_np.argmax(1)==3]
        X4 = X[y_np.argmax(1)==4]
        X5 = X[y_np.argmax(1)==5]
        X6 = X[y_np.argmax(1)==6]
        X7 = X[y_np.argmax(1)==7]
        plt.scatter(X2[:, 0], X2[:, 1], s=20, edgecolors='black', facecolor='None',
                    marker='s', linewidths=0.2)
        plt.scatter(X3[:, 0], X3[:, 1], s=20, edgecolors='gray', facecolor='None',
                    marker='s', linewidths=0.2)
        plt.scatter(X4[:, 0], X4[:, 1], s=20, edgecolors='m', facecolor='None',
                    marker='s', linewidths=0.2)
        plt.scatter(X5[:, 0], X5[:, 1], s=20, edgecolors='darksalmon', facecolor='None',
                    marker='s', linewidths=0.2)
        plt.scatter(X6[:, 0], X6[:, 1], s=20, edgecolors='tan', facecolor='None',
                    marker='s', linewidths=0.2)
        plt.scatter(X7[:, 0], X7[:, 1], s=20, edgecolors='olivedrab', facecolor='None',
                    marker='s', linewidths=0.2)
    plt.xlim([plot_min, plot_max])
    plt.ylim([plot_min, plot_max])
    plt.xlabel("x")
    plt.ylabel('y')
    
    margin = 0.01
    #rect = matplotlib.patches.Rectangle((-margin, -margin), 1.0+2*margin, 1.0+2*margin, 
    #                                    linewidth=1.5, color='white', fill=False)
    #ax.add_patch(rect)
    
    plt.gca().set_aspect('equal', adjustable='box')
    plt.savefig('two_moons_paper/{}_{:.1f}_{:.1f}_iters={}_max_prob={}.pdf'.format(
        name, plot_min, plot_max, n_iters, max_prob), transparent=True)
    plt.clf()
    #plt.show()

In [33]:
class MoonsDataset:
    def __init__(self, dim=2):
        self.dim = dim

    def _generate_moons(self, n_samples=2000, shuffle=True, noise=.02, x_lat_mult=0, x_long_mult=0):
        X, y = datasets.make_moons(n_samples=n_samples, shuffle=shuffle, noise=noise)
        # Rescale and shift the dataset to better fit into zero-one box
        X = (X + 1.6) / 4
        X[:, 0] = X[:, 0] - 0.035
        X[:, 1] = (X[:, 1] - 0.17) * 1.75
        X[:, 0] = X[:, 0] + x_lat_mult
        X[:, 1] = X[:, 1] + x_long_mult
        return X, y

    def two_moons(self, n_samples=2000, shuffle=True, noise=.02,
                  testing_samples=400, testing_shuffle=True, testing_noise=.02):
        X, y = self._generate_moons(n_samples, shuffle, noise)
        y = tf.one_hot(y, self.dim)
        X = tf.convert_to_tensor(X)
        X_test, y_test = self._generate_moons(testing_samples, testing_shuffle, testing_noise)
        return X, y, X_test, y_test

    def two_set_two_moons(self, n_samples=2000, shuffle=True, noise=.02,
                          testing_samples=400, testing_shuffle=True, testing_noise=.02, position='horizontal'):
        X, y = self._generate_moons(n_samples, shuffle, noise)

        X_test, y_test = self._generate_moons(testing_samples, testing_shuffle, testing_noise)

        if position == 'horizontal':
            X_extra, y_extra = self._generate_moons(n_samples, shuffle, noise, 1, 0)
            X_test_extra, y_test_extra = self._generate_moons(testing_samples, testing_shuffle, testing_noise, 1, 0)
        elif position == 'vertical':
            X_extra, y_extra = self._generate_moons(n_samples, shuffle, noise, 0, 1)
            X_test_extra, y_test_extra = self._generate_moons(testing_samples, testing_shuffle, testing_noise, 0, 1)
        else:
            raise Exception("Unknown two moons command")

        X = np.append(X, X_extra, axis=0)
        y = np.append(y, y_extra, axis=0)
        X_test = np.append(X_test, X_test_extra, axis=0)
        y_test = np.append(y_test, y_test_extra, axis=0)

        y = tf.one_hot(y, self.dim)
        X = tf.convert_to_tensor(X)

        return X, y, X_test, y_test

    def four_set_two_moons(self, n_samples=2000, shuffle=True, noise=.02,
                           testing_samples=400, testing_shuffle=True, testing_noise=.02):
        X, y = self._generate_moons(n_samples, shuffle, noise)

        X_test, y_test = self._generate_moons(testing_samples, testing_shuffle, testing_noise)

        for x_i in (0, 1):
            for y_i in (0, 1):
                if x_i == 0 and y_i == 0:
                    continue
                else:
                    X_extra, y_extra = self._generate_moons(n_samples, shuffle, noise, x_i, y_i)
                    X_test_extra, y_test_extra = self._generate_moons(testing_samples, testing_shuffle,
                                                                      testing_noise, x_i, y_i)
                    if self.dim == 8:
                        if x_i == 1 and y_i == 0:
                            y_extra = [2 if i == 1 else 3 for i in y_extra]
                            y_test_extra = [2 if i == 1 else 3 for i in y_test_extra]
                        if x_i == 0 and y_i == 1:
                            y_extra = [4 if i == 1 else 5 for i in y_extra]
                            y_test_extra = [4 if i == 1 else 5 for i in y_test_extra]
                        if x_i == 1 and y_i == 1:
                            y_extra = [6 if i == 1 else 7 for i in y_extra]
                            y_test_extra = [6 if i == 1 else 7 for i in y_test_extra]
                    X = np.append(X, X_extra, axis=0)
                    y = np.append(y, y_extra, axis=0)
                    X_test = np.append(X_test, X_test_extra, axis=0)
                    y_test = np.append(y_test, y_test_extra, axis=0)
        y = tf.one_hot(y, self.dim)
        X = tf.convert_to_tensor(X)
        return X, y, X_test, y_test

In [34]:
moons = MoonsDataset()
X, y, X_test, y_test = moons.two_moons()

In [35]:
n_iter = 400

In [36]:
callbacks = [
            CustomHistory()
        ]

In [37]:
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [38]:
if radial:
    H = model.fit(x=X, y=y, epochs=n_iter, callbacks=callbacks, batch_size=2000, shuffle=False)
else:
    H = model.fit(x=X, y=y, epochs=n_iter, batch_size=2000, shuffle=False)

Train on 2000 samples
Epoch 1/400
2000/2000 [==============================] - 1s 414us/sample - loss: 0.6871 - accuracy: 0.5000
Epoch 2/400
2000/2000 [==============================] - 0s 6us/sample - loss: 0.6680 - accuracy: 0.5000
Epoch 3/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.6523 - accuracy: 0.8415
Epoch 4/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.6362 - accuracy: 0.8295
Epoch 5/400
2000/2000 [==============================] - 0s 5us/sample - loss: 0.6168 - accuracy: 0.8265
Epoch 6/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.5931 - accuracy: 0.8385
Epoch 7/400
2000/2000 [==============================] - 0s 5us/sample - loss: 0.5732 - accuracy: 0.8490
Epoch 8/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.5671 - accuracy: 0.8510
Epoch 9/400
2000/2000 [==============================] - 0s 5us/sample - loss: 0.5563 - accuracy: 0.8515
Epoch 10/400
2000/2000 [=======

Epoch 79/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.3247 - accuracy: 0.8905
Epoch 80/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.3237 - accuracy: 0.8930
Epoch 81/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.3219 - accuracy: 0.8940
Epoch 82/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.3205 - accuracy: 0.8915
Epoch 83/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.3190 - accuracy: 0.8915
Epoch 84/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.3174 - accuracy: 0.8935
Epoch 85/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.3161 - accuracy: 0.8945
Epoch 86/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.3147 - accuracy: 0.8910
Epoch 87/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.3134 - accuracy: 0.8895
Epoch 88/400
2000/2000 [======================

Epoch 156/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.2525 - accuracy: 0.8955
Epoch 157/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.2526 - accuracy: 0.8965
Epoch 158/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.2519 - accuracy: 0.8955
Epoch 159/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.2505 - accuracy: 0.8965
Epoch 160/400
2000/2000 [==============================] - 0s 5us/sample - loss: 0.2487 - accuracy: 0.8960
Epoch 161/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.2490 - accuracy: 0.8955
Epoch 162/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.2480 - accuracy: 0.8985
Epoch 163/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.2463 - accuracy: 0.8965
Epoch 164/400
2000/2000 [==============================] - 0s 5us/sample - loss: 0.2447 - accuracy: 0.8970
Epoch 165/400
2000/2000 [============

Epoch 233/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.0629 - accuracy: 1.0000
Epoch 234/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.0594 - accuracy: 1.0000
Epoch 235/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.0590 - accuracy: 1.0000
Epoch 236/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.0569 - accuracy: 1.0000
Epoch 237/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.0568 - accuracy: 1.0000
Epoch 238/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.0543 - accuracy: 1.0000
Epoch 239/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.0540 - accuracy: 1.0000
Epoch 240/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.0518 - accuracy: 1.0000
Epoch 241/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.0519 - accuracy: 1.0000
Epoch 242/400
2000/2000 [============

Epoch 310/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.0223 - accuracy: 1.0000
Epoch 311/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.0221 - accuracy: 1.0000
Epoch 312/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.0219 - accuracy: 1.0000
Epoch 313/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.0218 - accuracy: 1.0000
Epoch 314/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.0216 - accuracy: 1.0000
Epoch 315/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.0214 - accuracy: 1.0000
Epoch 316/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.0213 - accuracy: 1.0000
Epoch 317/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.0211 - accuracy: 1.0000
Epoch 318/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.0210 - accuracy: 1.0000
Epoch 319/400
2000/2000 [============

Epoch 387/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.0136 - accuracy: 1.0000
Epoch 388/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.0136 - accuracy: 1.0000
Epoch 389/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.0135 - accuracy: 1.0000
Epoch 390/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.0134 - accuracy: 1.0000
Epoch 391/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.0134 - accuracy: 1.0000
Epoch 392/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.0133 - accuracy: 1.0000
Epoch 393/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.0132 - accuracy: 1.0000
Epoch 394/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.0132 - accuracy: 1.0000
Epoch 395/400
2000/2000 [==============================] - 0s 4us/sample - loss: 0.0131 - accuracy: 1.0000
Epoch 396/400
2000/2000 [============

In [39]:
model.save('two_moons_paper/100d_100d_relu_400_iter_mact_int_zeros_new_fit/model.h5')

In [45]:
name = "100d_100d_relu_400_iter_mact_int_zeros_new_fit/paper_vis"
plot(model, 0.0, 1.0, max_prob=True, n_iters=400, name=name, MAct=True)
plot(model, -2.0, 3.0, max_prob=True, n_iters=400, name=name, MAct=True)
plot(model, -5.0, 6.0, max_prob=True, n_iters=400, name=name, MAct=True)
plot(model, -10.0, 10.0, max_prob=True, n_iters=400, name=name, MAct=True)

<Figure size 432x288 with 0 Axes>

In [46]:
counter = 0
info_list = []
if radial:
    for epoch, train_loss, train_acc, b, c in zip(range(n_iter+1), H.history["loss"], H.history["accuracy"], H.history['b'], H.history['c']):
        if int(epoch) % 100 == 0:
            entry = "Epoch {}, loss: {}, acc: {}, b: {}, c: {}".format(epoch, train_loss, train_acc, b, c)
            info_list.append(entry)
    entry = "Epoch {}, loss: {}, acc: {}".format(n_iter, H.history["loss"][-1], H.history["accuracy"][-1], H.history['b'][-1], H.history['c'][-1])
else:
    for epoch, train_loss, train_acc in zip(range(n_iter+1), H.history["loss"], H.history["accuracy"]):
        if int(epoch) % 100 == 0:
            entry = "Epoch {}, loss: {}, acc: {}".format(epoch, train_loss, train_acc)
            info_list.append(entry)
    entry = "Epoch {}, loss: {}, acc: {}".format(n_iter, H.history["loss"][-1], H.history["accuracy"][-1])
info_list.append(entry)

In [47]:
file_name = "two_moons_paper/{}.csv".format(name + '_metrics')
with open(file_name, 'w', newline='') as myfile:
    wr = csv.writer(myfile, delimiter="\n")
    wr.writerow(info_list)